# This notebook is to help automatically import parquet schema to hive

Below is import of all needed dependencies. And in this sell you should pass path where parquet files located. 

In [1]:
import os

Here is creating of spark context with hive support.

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Import parquet schema to hive").config("hive.metastore.uris", "thrift://hive:9083").enableHiveSupport().getOrCreate()

Define function below for getting sql script needed for creating table in hive using dataframe types as columns to table

In [3]:
def getCreateTableScript(databaseName, tableName, path, df):
    cols = df.dtypes
    createScript = "CREATE EXTERNAL TABLE IF NOT EXISTS " + databaseName + "." + tableName + "("
    colArray = []
    for colName, colType in cols:
        colArray.append(colName.replace(" ", "_") + " " + colType)
    createColsScript =   ", ".join(colArray )
    
    script = createScript + createColsScript + ") STORED AS PARQUET LOCATION '" + path + "'"
    print(script)
    return script
    

In [4]:
#define main function for creating table where arqument 'path' is path to parquet files 
def createTable(databaseName, tableName, path): 
    df = spark.read.parquet(path)
    sqlScript = getCreateTableScript(databaseName, tableName, path, df)
    spark.sql(sqlScript)

## One file example

In [5]:
# Set path where concrete parquet file located.
my_parqute_file_path = os.path.join('v3io://users/admin/examples/userdata1.parquet')
createTable("default","table_from_single_file2",my_parqute_file_path)

CREATE EXTERNAL TABLE IF NOT EXISTS default.table_from_single_file2(registration_dttm timestamp, id int, first_name string, last_name string, email string, gender string, ip_address string, cc string, country string, birthdate string, salary double, title string, comments string) STORED AS PARQUET LOCATION 'v3io://users/admin/examples/userdata1.parquet'


## One folder example for spark output job

In [6]:
# Set path where parquet folder with parquet files inside located.
folder_path = os.path.join('v3io://users/admin/examples/spark-output/*')
createTable("default","table_from_dir2",folder_path)

CREATE EXTERNAL TABLE IF NOT EXISTS default.table_from_dir2(registration_dttm timestamp, id int, first_name string, last_name string, email string, gender string, ip_address string, cc string, country string, birthdate string, salary double, title string, comments string) STORED AS PARQUET LOCATION 'v3io://users/admin/examples/spark-output/*'


# Multiple files and folders example

Write here name of database and path to folder where all parquet files (or folders with them) located. Database should be created.
In this sell code goes over all files and dirs in provided path and using them for creating table.
File should be ended with .parquet format
Directory (in which stored parquet files) should be started with "."
Name of directory or file will be name of table.

In [7]:
databaseName = "default"
filepath = "/v3io/users/admin/examples/multiple-parquet-files"

for fileOrDir in os.listdir(filepath):
    if fileOrDir.endswith(".parquet") :
        createTable(databaseName, fileOrDir.split(".parquet")[0], filepath.replace("/v3io/", "v3io://", 1) + "/" + fileOrDir)
    elif not fileOrDir.startswith(".") :
        createTable(databaseName, fileOrDir, filepath.replace("/v3io/", "v3io://", 1) + "/" + fileOrDir + "/*")



CREATE EXTERNAL TABLE IF NOT EXISTS default.userdata1(registration_dttm timestamp, id int, first_name string, last_name string, email string, gender string, ip_address string, cc string, country string, birthdate string, salary double, title string, comments string) STORED AS PARQUET LOCATION 'v3io://users/admin/examples/multiple-parquet-files/userdata1.parquet'
CREATE EXTERNAL TABLE IF NOT EXISTS default.userdata2(registration_dttm timestamp, id int, first_name string, last_name string, email string, gender string, ip_address string, cc string, country string, birthdate string, salary double, title string, comments string) STORED AS PARQUET LOCATION 'v3io://users/admin/examples/multiple-parquet-files/userdata2.parquet'
CREATE EXTERNAL TABLE IF NOT EXISTS default.userdata3(registration_dttm timestamp, id int, first_name string, last_name string, email string, gender string, ip_address string, cc string, country string, birthdate string, salary double, title string, comments string) STO

# Test how it works

In [11]:
# test how the tables were saved
#spark.sql("drop database test CASCADE")
spark.sql("show databases").show()
spark.sql("show tables in " + databaseName).show()

+------------+
|databaseName|
+------------+
|     default|
|        test|
+------------+

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|    test|                dir1|      false|
|    test|      table_from_dir|      false|
|    test|     table_from_dir2|      false|
|    test|table_from_single...|      false|
|    test|table_from_single...|      false|
|    test|           userdata1|      false|
|    test|           userdata2|      false|
|    test|           userdata3|      false|
+--------+--------------------+-----------+



In [ ]:
# test how saving to table works
tableName = "example1"
spark.sql("select * from " + databaseName + "." + tableName)

In [ ]:
spark.sql("drop table " + databaseName + ".example1")